In [ ]:
# import pandas as pd
# import numpy as np

In [ ]:
# df = pd.read_csv('Reviews.csv')

In [ ]:
# u_userid = df['UserId'].unique()
# u_productid = df['ProductId'].unique()

In [ ]:
# u_userid

In [ ]:
# u_productid

In [ ]:
# productid_index = {}
# for index,productid in enumerate(u_productid):
#     productid_index[productid] = index

In [ ]:
# df['UserId_ProductId'] = df.apply(lambda x:'%s_%s' % (x['UserId'],x['ProductId']),axis=1)

In [ ]:
# df.sort_values(by=['UserId_ProductId'],inplace=True)

In [ ]:
# df.reset_index(inplace=True)

In [ ]:
# with open('test.csv','w') as f:
#     header = ['UserID,ProductId']
#     dictionary = {'UserId':u_productid}
#     data_bak = [0]*len(u_productid)
#     writer = csv.writer(f)
#     for i in range(len(df)):
#         row = df.loc[i:i]
#         userid = row.iloc[0]['UserId']
#         productid = row.iloc[0]['ProductId']
#         score = row.iloc[0]['Score']
#         if userid not in dictionary:
#             for key,value in dictionary.items():
#                 data = key
#                 for v in value:
#                     data += ','+str(v)
#             f.write(data+'\n')
#             dictionary = {}
#             dictionary[userid] = data_bak
#             dictionary[userid][productid_index[productid]] = score
#         else:
#             dictionary[userid][productid_index[productid]] = score


## preprocessing - (to3tuple)

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('Reviews_sa.csv')
df['score_with_sa'] = df['Score']-df['sa_compound_score']
df

,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,sa_compound_score,score_with_sa
0,0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,0.9441,4.0559
1,1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,-0.5664,1.5664
2,2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,0.8265,3.1735
3,3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,0.0000,2.0000
4,4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,0.9468,4.0532
5,5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...,0.8830,3.1170
6,6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...,0.9346,4.0654
7,7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...,0.9487,4.0513
8,8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...,0.6369,4.3631
9,9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...,0.8313,4.1687


In [3]:
def encode_df(df):
def get_vip(df):
    return df[df.groupby('UserId').UserId.transform(len) > 1]

In [5]:
## generate ratings
users = df['UserId'].unique()
items = df['ProductId'].unique()
users_dict = {users[i]:i for i in range(len(users))}
items_dict = {items[i]:i for i in range(len(items))}
intUserId = df.apply(lambda x: users_dict[x['UserId']], axis=1)
intItemId = df.apply(lambda x: items_dict[x['ProductId']], axis=1)
ratings = pd.concat([intUserId,intItemId,df['Score']], axis=1)
ratings.to_csv('Ratings.csv', index=False, header=False)

In [6]:
#remove users appears once and generate ratings
df_vip = get_vip(df)
users = df_vip['UserId'].unique()
items = df_vip['ProductId'].unique()
users_dict = {users[i]:i for i in range(len(users))}
items_dict = {items[i]:i for i in range(len(items))}
intUserId = df_vip.apply(lambda x: users_dict[x['UserId']], axis=1)
intItemId = df_vip.apply(lambda x: items_dict[x['ProductId']], axis=1)
ratings = pd.concat([intUserId,intItemId,df_vip['Score']], axis=1)
ratings.to_csv('Ratings_vip.csv', index=False, header=False)

In [7]:
df.size

7389902

In [8]:
df_vip.size

5109819

In [10]:
# df= df[df.groupby('UserId').UserId.transform(len) > 1]
users = df_vip['UserId'].unique()
items = df_vip['ProductId'].unique()
users_dict = {users[i]:i for i in range(len(users))}
items_dict = {items[i]:i for i in range(len(items))}
intUserId = df_vip.apply(lambda x: users_dict[x['UserId']], axis=1)
intItemId = df_vip.apply(lambda x: items_dict[x['ProductId']], axis=1)
ratings = pd.concat([intUserId,intItemId,df_vip['score_with_sa']], axis=1)
ratings.to_csv('Ratings_vip_sa.csv', index=False, header=False)

compute rating


In [13]:
#moderate user scores based on user input and group input and generate ratings
df_vip['AvgUserScore'] = df_vip['Score'].groupby(df_vip['UserId']).transform('mean')
df_vip['AvgProductScore'] = df_vip['Score'].groupby(df_vip['ProductId']).transform('mean')
df_vip['ModeratedScore'] = 0.75*df_vip['AvgUserScore'] + 0.25*df_vip['AvgProductScore']
users = df_vip['UserId'].unique()
items = df_vip['ProductId'].unique()
users_dict = {users[i]:i for i in range(len(users))}
items_dict = {items[i]:i for i in range(len(items))}
intUserId = df_vip.apply(lambda x: users_dict[x['UserId']], axis=1)
intItemId = df_vip.apply(lambda x: items_dict[x['ProductId']], axis=1)
ratings = pd.concat([intUserId,intItemId,df_vip['ModeratedScore']], axis=1)
ratings.to_csv('Ratings_vip_moderateS.csv', index=False, header=False)

C:\Users\lyuch\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\lyuch\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\lyuch\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t